<p align="center">
    <img src="https://github.com/jessepisel/energy_analytics/blob/master/EA_logo.jpg?raw=true" width="220" height="240" />

</p>

# Removal of 'strip noise' in radio-echo sounding data using WavCycleGAN

## Freshman Research Initiative Energy Analytics CS 309

#### Written by: Akash Aedavelli, Akif Abidi, Alan Goff, Nicholas Khami, Omar Imtiaz


## Introduction

One of the most commonly used tecniques in glaciological exploration is radio-echo sounding (RES). The data provided from RES is incredibly helpful to researchers when it comes to determining ice thickness and understanding subglacial conditions. However, RES data has a flaw in that interference from both the internal instruments used to acquire the data and environmental conditions cause 'strip noise' to appear in the acquired radargrams. Strip noise can best be characterized as vertical, horizontal, and diagnol stripes that appear in RES records. It's very problematic for researchers as it makes it substantially more difficult to interpret the data. With our research, we seek to determine if this 'strip-noise' can be removed from RES records using WavCycleGAN. 

## Related Works

Due to the importance of RES data, the removal of strip-noise has been the subject of many research papers over the past two decades. There have been several attempts to remove strip-noise using a variety of filtering techniques, the most recent of which was 2-D DFT Filtering (Wang and other, 2019). However, the drawback of using filtering techniques is that they can only be shown to work using synthetically created and simplified mathematical models. Even if the filtering technique is proven via the model, the RES data may still have unexpected properties that cause strip-noise to remain or the data to become overprocessed after the filtering is applied. 

As best as our research team has been able to see, no-one has attempted to use deep learning based methods to remove the strip-noise in RES data. We hypothesize that this is because a paired dataset of noisy and clean RES records does not exist, therefore necessitating that researchers would need to add synthetic noise to relatively clean images. At the point where one is adding synthetic noise, a deep-learning based approach becomes no better than a filtering approach as it has the same flaws in there potentially being unexpected properties in the data that it does not address.

Understanding this, we seek to use an unsupervised learning appprach that uses unpaired clean and noisy RES records. Our plan is then to create an unsupervised learning flow to train a nueral network using the unpaired data. This should allow us to effectively remove strip-noise without requiring paired data. This technique is the root of 'CycleGAN' and has previously been employed to successfully denoise sattelite images (Song and others, 2020). 

Additionally, further following discoveries made denoising sattellite images (Song and others, 2020) our group intends to use wavelet subband images rather than the entire image domain when training our model. Since only certain wavelet subbands are going to contain strip-noise, our model should be able to ignore non-relevant bands and thus preserve the information within them. This technique constitutes the 'Wav' in 'WavCycleGAN.' 

## Procedure for generating wavelet subband images 

WavCycleGAN uses wavelet subband images that are generated via a 2D Daubhechies-3 wavelet transofmration that decomposes an image into subbands that such that there is a horizontal detail approdimation, vertical detail approximation, and diagnol detail approximation. The formulas by which this is done are not specifically relevant to this research, beacuse they are done by imported packages.  

Our eventual workflow will be to remove and identify noise from the subband images and then subtract those noise patterns from the original image to clean it. 

## Theoretical Network Architecture 

The wavCycleGAN architecture we intend to use is fundamentally the same as the standard CycleGAN architecture, as pictured below:  

<p align="center">
    <figure>
        <img src="https://www.nicholaskhami.com/images/architecture.png" width="401" height="196" />
        <figcaption style="text-align:center;font-style:italic;font-size:10px;">Architecture of WavCycleGAN (Song and others, 2020)</figcaption>
    </figure>
</p>

Teh generator G<sub>Θ</sub> trains against the discriminator ϕ<sub>Φ</sub> to produce clean versions of subband images while the generator F<sub>Λ</sub> trains against the discriminator ψ<sub>Ξ</sub> to produce noisy versions of subband images. During training, to ensure that the generators do not deviate too far from the original image, the CycleGAN workflow makes use of adversarial loss <i>ℓ<sub>GAN</sub></i> that can be described as follows: 

$$
\ell_{GAN}(\Theta,\Phi) = \mathbb{E}_{x~P_\mathcal{X}}[log_{\varphi\phi}(x)] + \mathbb{E}_{y~P_\mathcal{Y}}[log_{1 - \varphi\phi}(G_\Theta(y))]
$$

As the discriminators and generators train each other, they continually attempt to minimimize loss so as to reduce artifacts. Cycle-consistency loss is also used to make sure that the generator maintains important features outher than noise. The cycle consistency loss function is defined as follows: 

In [9]:
import numpy as np
import netCDF4 as nc
from PIL import Image
from os import listdir, getcwd
from os.path import isdir, join, dirname, split

In [2]:
path = join(dirname(getcwd()),'data')

In [3]:
dirs = listdir(path)

In [6]:
path = join(dirname(getcwd()),'data')
dirs = listdir(path)
for folder in dirs:
    tmp_path = join(path, folder)
    if isdir(tmp_path):
        files = listdir(tmp_path)
        for file in files:
            if file.endswith(".nc"):
                create_radargram(join(tmp_path, file))

In [8]:
def create_radargram(file_path):
    _, file_name = split(file_path)
    file_name = file_name[:-3]
    ds = nc.Dataset(file_path)
    low_gain = np.asarray(ds['amplitude_low_gain'])
    high_gain = np.asarray(ds['amplitude_high_gain'])
    low_gain = 255 * (low_gain-np.amin(low_gain))/(np.amax(low_gain) - np.amin(low_gain))
    high_gain = 255 * (high_gain-np.amin(high_gain))/(np.amax(high_gain) - np.amin(high_gain))
    Image.fromarray(low_gain.T).convert("L").save("images/"+file_name+"_LOW.png")
    high_img = Image.fromarray(high_gain.T).convert("L").save("images/"+file_name+"_HIGH.png")